In [ ]:
import requests
import geopandas as gpd
import pandas as pd
import geojson
import time
import os

In [ ]:
layer_endpoint = {
    'settlement' : 'count',
    'highway' : 'length',
    'railway' : 'length',
    'lake' : 'area',
    'river' : 'length',
    
}

layer_filter = {
    'settlement' : 'place in (city,town) and type:node',
    'highway' : 'highway in (motorway, trunk, primary) and type:way',
    'railway' : 'railway=rail and type:way',
    'lake' : '(water in (lagoon,lake,reservoir) or landuse=reservoir) and type:way',
    'river' : 'waterway in (riverbank,river) and type:way',
    
}

indicator_name = {
    'saturation' : 'MappingSaturation',
    #'currentness' : 'Currentness', => not currently supported when querying oqt with data from ohsome given
}


list_layers = list(layer_endpoint.keys())
list_indicators = list(indicator_name.keys())

In [ ]:
def geometry_json_to_polygon(region_json):
    
    
    if region_json['geometry']['type'] == 'MultiPolygon':
        region_polygon = ''
        for new_poly in region_json['geometry']['coordinates']:
            region_polygon = region_polygon + '|' + str(new_poly[0])
        region_polygon = region_polygon.replace('[','').replace(']','') 
        region_polygon = region_polygon[1:]
    else:
        region_polygon = str(region_json['geometry']['coordinates']).replace('[','').replace(']','')
        
    return(region_polygon)
        
        

In [ ]:
def ohsome_api_call(region_polygon, query_endpoint, query_filter):
    
    url = 'https://api.ohsome.org/v1/elements/' + query_endpoint
    data = {
        "bpolys": region_polygon,
        "format": "json", 
        "time": "2008-01-01/2022-08-01/P1M", 
        "filter": query_filter}

    response = requests.post(url, data=data)

    layer_response = response.json()


    osm_layer = {}
    osm_layer['name'] = ''
    osm_layer['description'] = ''
    osm_layer['data'] = {}
    if response.ok:
        osm_layer['data']['result'] = layer_response['result']
    
    return(osm_layer)

In [ ]:
def oqt_api_call(region_json, osm_layer, indicator_name):
    
    url = "https://oqt.ohsome.org/api/indicator"

    parameters = {
        "name": indicator_name,
        "bpolys": region_json,
        "layer": osm_layer,
        "includeSvg": False,  # Optional
        "includeHtml": False,  # Optional
        "flatten": True,  # Optional
    }

    response = requests.post(url, json=parameters)
    
    if response.ok:
        result_json = response.json()
        result_label = result_json['properties']['result.label']
        result_score = result_json['properties']['result.value']
        
        if result_label == 'green':
            result_label = 'high'
        elif result_label == 'yellow':
            result_label = 'medium'
        elif result_label == 'red':
            result_label = 'low'

        return([result_label,result_score])
    else:
        return(['undefined',0])


In [ ]:

def compute_report(input_df):


    t0 = time.time()
    
    input_geojson = geojson.loads(input_df.to_json())

    result_label_list = {}
    result_value_list = {}

    for layer in list_layers:
        result_label_list[layer] = {}
        result_value_list[layer] = {}

        for indicator in list_indicators:
            result_label_list[layer][indicator] = []
            result_value_list[layer][indicator] = []


    for i in range(0,input_df.shape[0]):


        region_json = input_geojson['features'][i]
        region_polygon = geometry_json_to_polygon(region_json)


        for layer in list_layers:

            osm_layer = ohsome_api_call(region_polygon, layer_endpoint[layer], layer_filter[layer])

            for indicator in list_indicators:
                indicator_score = oqt_api_call(region_json, osm_layer, indicator_name[indicator])

                result_label_list[layer][indicator].append(indicator_score[0])
                result_value_list[layer][indicator].append(indicator_score[1])


        print(str(i+1) + '/' + str(input_df.shape[0])  + ' subdivisions ...')
        

    for layer in list_layers:
        for indicator in list_indicators:

            layer_indicator_name = layer + '_' + indicator

            input_df[layer_indicator_name + '_label'] = result_label_list[layer][indicator]
            #input_df.loc[input_df[layer_indicator_name + '_label'] == 'undefined', layer_indicator_name + '_label'] = 'red'
 
            input_df[layer_indicator_name + '_value'] = result_value_list[layer][indicator]
            input_df[layer_indicator_name + '_value'].fillna(0, inplace = True)
            
    output_df = input_df.copy()
    
    print('done')
    dt = time.time()   - t0
    print('Execution time: {0:.2f}s'.format(dt))
    
    return(output_df)
        
        

In [ ]:
path = os.getcwd() + '/cdema_countries/admin1/'
#path = os.getcwd() + '/cesdrr_countries/admin1/'


input_file_list = []

for file in os.listdir(path):
    if os.path.splitext(file)[1] == '.geojson':
        input_file_list.append(file)

In [ ]:
for file in input_file_list:


    input_file_name = './cdema_countries/admin1/' + file
    output_file_name = './cdema_countries/admin1-results/' + file[:-8] + '_results.geojson'
    
    #input_file_name = './cesdrr_countries/admin1/' + file
    #output_file_name = './cesdrr_countries/results/' + file[:-8] + '_results.geojson'

    print(input_file_name)

    input_df = gpd.read_file(input_file_name)
    rapport_df = compute_report(input_df)
    rapport_df.to_file(output_file_name, driver="GeoJSON")  

    print(output_file_name)

    print('------')